<a href="https://colab.research.google.com/github/LuigiSigillo/CartoonGAN/blob/main/CartoonGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#TODO